# Imports

In [1]:
import sys
import nltk
import numpy as np
import pandas as pd
import pickle
import os

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn import metrics
from sklearn.metrics import classification_report

sys.path.append(".")
sys.path.append("..")

# Set Params

In [2]:
# Input
text_columns = "description"
# Supported datasets:
# summary
# description

# Output
column_to_predict = "urgency"
# Supported datasets:
# impact
# urgency

# Classifier
classifier = "SVM"
# Supported algorithms:
# DT
# NB
# SVM

# Removes stop words from processed text
remove_stop_words = True
stop_words_lang = 'english'

# Word stemming using NLTK
use_stemming = False

# fit_prior: whether to learn class prior probabilities or not.
# If false, a uniform prior will be used.
if use_stemming:
    fit_prior = False
else:
    fit_prior = True

# Load Data Set

In [3]:
dfTickets = pd.read_csv(
    './dataset/tickets_1.csv',
    engine='python',
    dtype=str
)  

# Turn Strings into Lower Case

In [4]:
dfTickets.apply(lambda x: x.astype(str).str.casefold())

,summary,description,impact,urgency,priority
0,"""back to top"" button is on the wrong place",!image-2019-01-23-17-03-59-735.png!,3,3,3
1,"""code master maintenance"" link is not working",see attached screenshot.,2,1,2
2,"""x"" should not be there",see attached screenshot.,3,3,3
3,(next phase) missing content in form but submi...,"there is no form appear in this services, howe...",1,1,1
4,(phase 2) code maintenance - missing 'effectiv...,after *order* field there is should be '*effec...,1,3,1
...,...,...,...,...,...
1571,validation message for every field,please provide the error message for every man...,3,3,3
1572,wrong lov for relationship,"there is no ""others"" option under ""i am the de...",2,3,2
1573,wrong error message,please correct the below error message\n\n!ret...,3,3,3
1574,â€�view allâ€� button is not working,active application service:â \n\n!image-2019-0...,3,3,3


# Split Data Set into Train and Test Data

In [5]:
labelData = dfTickets[column_to_predict]
data = dfTickets[text_columns]

# Split dataset into training and testing data with 80:20 ratio
train_data, test_data, train_labels, test_labels = train_test_split(data, labelData, test_size=0.2)

# Extract Features from the Text

In [6]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        stemmer = SnowballStemmer("english", ignore_stopwords=True)
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

## Count Vectorizer

In [7]:
# Count Vectorizer = tokenizer
# Convert a collection of text documents to a matrix of token counts
if remove_stop_words:
    count_vect = CountVectorizer(stop_words=stop_words_lang)
elif use_stemming:
    count_vect = StemmedCountVectorizer(stop_words=stop_words_lang)
else:
    count_vect = CountVectorizer()

vectorized_data = count_vect.fit_transform(train_data)

## Use TF-IDF

In [8]:
tfidf = TfidfTransformer()
features = tfidf.fit_transform(vectorized_data)

# Use A Pipeline to Preprocess Data and Train Classifier

In [9]:
if classifier == "NB":
    clf = Pipeline([
        ('vect', count_vect),
        ('tfidf', TfidfTransformer()),
        ('samp',RandomOverSampler()),
        ('clf', MultinomialNB(fit_prior=fit_prior))
    ])
elif classifier == "DT":
    clf = Pipeline([
        ('vect', count_vect),
        ('tfidf', TfidfTransformer()),
        ('samp',RandomOverSampler()),
        ('clf', DecisionTreeClassifier())
    ])
elif classifier == "SVM":
    clf = Pipeline([
        ('vect', count_vect),
        ('tfidf', TfidfTransformer()),
        ('samp',RandomOverSampler()),
        ('clf', SVC(kernel='linear'))
    ])
clf = clf.fit(train_data, train_labels)

# Use GridSearchCV to Find the Best Params Set

In [10]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False)
}
# Create GS instance by passing the classifier, parameters and n_jobs=-1 (use multiple cores from user machine)
clf = GridSearchCV(clf, parameters, cv=10, n_jobs=-1)
clf = clf.fit(train_data, train_labels)

# Save Model to Pickle

In [11]:
pickle.dump(clf, open('./model/'+classifier+'_'+text_columns+'_'+column_to_predict+'_model.pickle',"wb"))

# Evaluate the Model

In [12]:
# Score and evaluate model on test data using model
predicted = clf.predict(test_data)

prediction_acc = np.mean(predicted == test_labels)
print("Confusion:")
print(metrics.confusion_matrix(test_labels, predicted))
print("Mean: " + str(prediction_acc))

Confusion:
[[  7   8  23]
 [  3  25  41]
 [ 12  24 173]]
Mean: 0.6487341772151899


# Print the Classification report

In [13]:
print(classification_report(test_labels, predicted,
                            target_names=np.unique(test_labels)))

              precision    recall  f1-score   support

           1       0.32      0.18      0.23        38
           2       0.44      0.36      0.40        69
           3       0.73      0.83      0.78       209

    accuracy                           0.65       316
   macro avg       0.50      0.46      0.47       316
weighted avg       0.62      0.65      0.63       316

